# AI Search with Vector Compression

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import json
import time
import numpy as np

# Load environment variables from .env file
load_dotenv(override=True)
%load_ext autoreload
%autoreload 2

In [3]:
# %pip install mercury

In [4]:
search_service_endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
search_service_key = os.environ["AZURE_SEARCH_KEY"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 3072))
embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")

In [5]:
from azure.search.documents.indexes import SearchIndexClient
search_index_client = SearchIndexClient(
search_service_endpoint, AzureKeyCredential(search_service_key))

In [11]:
from indexes import indexes_config
indexes = list(indexes_config.keys())
print(indexes)
search_queries = json.load(open("test_data.json"))
search_queries = search_queries["data"]
print(search_queries[0])

['py_embeddings']
What is the main plot of the first Harry Potter book?


In [ ]:
def bytes_to_kb(bytes):
    return round(bytes / 1024, 4)

def bytes_to_mb(bytes):
    return round(bytes / (1024 * 1024), 4)

def find_storage_size_mb(index_name):
    response = search_index_client.get_index_statistics(index_name)
    return bytes_to_kb(response["storage_size"]), bytes_to_kb(response["vector_index_size"])

index_sizes = [(find_storage_size_mb(index), index) for index in indexes]
index_sizes.sort(key=lambda item: item[0][0], reverse=True)

for ((storage_size, vector_size), index_name) in index_sizes:
    print("*" * 40)
    print(f"Index Name: {index_name}\nStorage Size: {storage_size}KB\nVector Size: {vector_size}KB")

****************************************
Index Name: py_embeddings
Storage Size: 2568.2598MB
Vector Size: 102.4023MB


In [13]:
def search_docs(search_client, search_text):
    search_text = search_text.replace("\"", "")
    start = time.time()
    results = search_client.search(search_text=search_text, top=1)
    text = ""
    for res in results:
        text, score = res["chunk"], res["@search.score"]
    end = time.time()
    return text, score, end-start

search_client = SearchClient(endpoint=search_service_endpoint, index_name=indexes[0], credential=AzureKeyCredential(search_service_key))
df = pd.DataFrame()
# pick a random search_text from the dataset
number_of_trials = 10
results = []
for i in range(number_of_trials):
    # pick a random number from 0 to len(test_data)
    random = np.random.randint(0, len(search_queries))
    search_text = search_queries[random]
    text, score, tt = search_docs(search_client, search_text)
    score = float(score)
    tt = float(tt)
    results.append({"search_text": search_text, "search_results_text": text, "search_results_score": score, "search_results_time": tt})
df = pd.DataFrame(results)
df.head()

,search_text,search_results_text,search_results_score,search_results_time
0,What role does Albus Dumbledore play in the fi...,Harry Potter and the Chamber of Secrets by J...,3.464750,0.905596
1,"How does the relationship between Harry, Ron, ...",Harry Potter and the Chamber of Secrets by J...,5.032452,0.224479
2,What are the main themes in Harry Potter and t...,Harry Potter and the Chamber of Secrets by J...,4.741489,0.252667
3,What is the main plot of the first Harry Potte...,Harry Potter and the Chamber of Secrets by J...,4.655178,0.219640
4,What challenges does Harry face in his first y...,Harry Potter and the Chamber of Secrets by J...,5.100635,0.213800
